# Exercice : appliquer les méthodes sur le dataset Indian Diabete

Importation des bibbliothèques de bases utilisées :

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

Importation des données dans un tableau :

In [ ]:
t = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
# Affichage des 10 premiers éléments du tableau
t.head(10)

On remarque que certaines données dans le tableau sont erronnées dans le tableau car elles sont à 0 ce qui n'est pas normal.  
A l'aide d'une fonction replace_0, on remplace les 0 par une valeur aléatoire suivant la loi normale déterminé avec les valeurs de la moyenne et de l'écart-type.

In [ ]:
t = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
# créer une valeur aléatoire suivant la loi normale et les valeurs de la moyenne et de l'écart-type
def replace_0(df,col) :
    df1 = df.copy()
    df1[col] = df[col].replace(0,np.random.normal(df[col].mean(),df[col].std()))
    return df1

On remplace les 0 par ne valeur aléatoire suivant la loi normale :

In [ ]:
t = replace_0(t,'Glucose')
t = replace_0(t,'BloodPressure')
t = replace_0(t,'SkinThickness')
t = replace_0(t,'Insulin')
t = replace_0(t,'BMI')

On vérifie qu'il n'y a bien plus de 0.

In [ ]:
t[t.Glucose==0]
t[t.BloodPressure==0]
t[t.SkinThickness==0]
t[t.Insulin==0]
t[t.BMI==0]

Le remplacement a bien fonctionné.  
Création de l'apprentissage :

In [ ]:
#Caractéristiques
X = t.drop(['Outcome'], axis=1)
#Résultats
y = t.Outcome

In [ ]:
#Importation de méthode pour la séparation des ensembles
from sklearn.model_selection import train_test_split
#Séparation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
#Vérification du bon fonctionnement séparation entre l'apprentissage et le test
print(X_train.shape)
print(X_test.shape)

# Méthode de régression logistique

In [ ]:
#Importation de la méthode de régression logistique
from sklearn.linear_model import LogisticRegression
#Entrainement
lr = LogisticRegression()
lr.fit(X_train,y_train)
#Prédiction
y_lr = lr.predict(X_test)

In [ ]:
# Importation des méthodes de mesure de performances
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

In [ ]:
#Calcul du score
rf_score = accuracy_score(y_test, y_lr)
print(rf_score)

In [ ]:
#Matrice de confusion
print(confusion_matrix(y_test,y_lr))
pd.crosstab(y_test, y_lr, rownames=['Reel'], colnames=['Prediction'], margins=True)

In [ ]:
print(classification_report(y_test, y_lr))

On remarque une différence entre la qualité de prédiction des 0 (Pas de diabète) et la qualité de prédiction des 1 (diabète).

In [ ]:
t.Outcome.value_counts()

On remarque que cela est dû à un désiquilibre entre les valeurs 0 et 1 de la colonne Outcome dans la base de données.

# Méthode de Random Forests

Calcul du score de précision avec la méthode Random Forests.  
Dans cette partie nous utilisons à présent le tableau modifier sans les valeurs de BloodPressur à 0.

In [ ]:
#Importation de la méthode randon forests
from sklearn import ensemble
#Entrainement
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
#Prédiction
y_rf = rf.predict(X_test)
#Calcul du score
rf_score = accuracy_score(y_test, y_rf)
print(rf_score)

In [ ]:
#Matrice de confusion
cm = confusion_matrix(y_test, y_rf)
print(cm)
pd.crosstab(y_test, y_rf, rownames=['Reel'], colnames=['Prediction'], margins=True)

On obtient un score de variable entre 76% et 82% qui est peut correcte et meilleur que la méthode de régression logistique vu précédemment selon la sélection aléatoire choisis.

In [ ]:
print(classification_report(y_test, y_rf))

Comparé à la méthode précédente, on remarque que la différence entre la qualité de prédiction des 0 et des 1 est réduite.

Utilisation d'hyperparamètres :

Parmi les hyperparamètres de l'algorithme qui peuvent avoir un impact sur les performances, on a :

* n_estimators : le nombre d'arbres de décision de la forêt aléatoire
* min_samples_leaf : le nombre d'échantillons minimum dans une feuille de chaque arbre
* max_features : le nombre de caractéristiques à prendre en compte lors de chaque split

On va tester plusieurs combianaisons de paramètre pour obtenir la meilleur possible.

In [ ]:
from sklearn import model_selection
param_grid = {
              'n_estimators': [10, 100, 500],
              'min_samples_leaf': [1, 20, 50],
              'max_features': [1, 2, 4, 8]
             }
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)

In [ ]:
# Détermination du meilleur groupe de paramètres
rf_gs.fit(X_train, y_train)

In [ ]:
#Affichage du meilleur groupe de paramètre
print(rf_gs.best_params_)

In [ ]:
rf2 = rf_gs.best_estimator_
y_rf2 = rf2.predict(X_test)
rf_score = accuracy_score(y_test, y_rf2)
print(rf_score)
print(classification_report(y_test, y_rf2))

# XGBoost

Calcul du score de précision avec la méthode XGBoost.

In [ ]:
# Sous Jupyter, si xgboost n'est pas déjà installé
!pip install xgboost

In [ ]:
#Importation de la méthode
import xgboost as XGB
#Entrainement
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
#Prédiction
y_xgb = xgb.predict(X_test)
#Calcul du score
rf_score = accuracy_score(y_test, y_xgb)
print(rf_score)
#Matrice de confusion 
cm = confusion_matrix(y_test, y_xgb)
print(cm)
#Classification report
print(classification_report(y_test, y_xgb))

On obtient un score de précision de 75.32% ce qui est moins bien quz les scores précédents.

Analyse de l'importance des caractéristiques :

In [ ]:
importances = rf2.feature_importances_
indices = np.argsort(importances)
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caracteristiques')

Grâce à l'importance des caractèristiques, on remarque que le glucose joue un rôle très important dans l'identification d'un diabète.